# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [32]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [33]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [34]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7feb46d18a00>)

In [35]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [36]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [37]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [38]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [39]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [40]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [41]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [42]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [43]:
# Only look at US regions
df = df[(df['CountryName'] == 'United States') & (df['RegionName'].notnull())]

In [44]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
72306,United States,Alaska,United States__Alaska,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72307,United States,Alaska,United States__Alaska,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72308,United States,Alaska,United States__Alaska,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72309,United States,Alaska,United States__Alaska,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72310,United States,Alaska,United States__Alaska,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90415,United States,Wyoming,United States__Wyoming,2020-07-27,45.0,3.0,1.0,1.0,3.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0
90416,United States,Wyoming,United States__Wyoming,2020-07-28,69.0,3.0,1.0,1.0,3.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0
90417,United States,Wyoming,United States__Wyoming,2020-07-29,39.0,3.0,1.0,1.0,3.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0
90418,United States,Wyoming,United States__Wyoming,2020-07-30,58.0,3.0,1.0,1.0,3.0,2.0,1.0,1.0,3.0,2.0,2.0,2.0,2.0


In [45]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positivea
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [46]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [47]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [48]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [49]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 121.03904646267992
Test MAE: 131.26026845169318


In [50]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.19242687998242594
Day -6 NewCases 0.2138179052370295
Day -5 NewCases 0.05736538230446278
Day -3 NewCases 0.023319698648850497
Day -2 NewCases 0.14122245394695032
Day -1 NewCases 0.402786408062919
Day -25 C2_Workplace closing 16.83112580530535
Day -22 C5_Close public transport 1.8046454711037945
Day -18 C6_Stay at home requirements 0.38183273726443134
Day -5 H3_Contact tracing 7.467119677015083
Intercept 43.16729605914975


In [51]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [32]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [33]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 54.44881708233056
2020-08-02: 74.42053319285834
2020-08-03: 83.76505684808843
2020-08-04: 89.70637523148649
2020-08-05: 95.33469756114599
2020-08-06: 96.9450701122124
2020-08-07: 108.07686916546967
2020-08-08: 133.94341467368184
2020-08-09: 152.96280205953656
2020-08-10: 167.37680374208267
2020-08-11: 179.26167539326275
2020-08-12: 189.15207127751415
2020-08-13: 197.87956592451826
2020-08-14: 211.28988474415002
2020-08-15: 229.71761848618405
2020-08-16: 247.33771868586038
2020-08-17: 263.46509033671435
2020-08-18: 278.21115372025986
2020-08-19: 291.590246481266
2020-08-20: 304.63914555571694
2020-08-21: 319.6781540710706
2020-08-22: 336.78453324137763
2020-08-23: 354.27690773104433
2020-08-24: 371.49804776791416
2020-08-25: 388.15278728249575
2020-08-26: 404.1653747094015
2020-08-27: 420.1156324430059
2020-08-28: 429.7844343752716
2020-08-29: 445.0021961262279
2020-08-30: 461.5993438254635
2020-08-31: 478.8197330618992

Predicting for Afghanistan_

2020-08-25: 2858.2077660941745
2020-08-26: 2882.7245029225674
2020-08-27: 2874.3974911801065
2020-08-28: 2873.494947183805
2020-08-29: 2918.2076923533236
2020-08-30: 2991.1903869820185
2020-08-31: 3075.7903960442127

Predicting for Azerbaijan__nan
2020-08-01: 2483.0186354558
2020-08-02: 3220.3107457551355
2020-08-03: 3640.1973338496123
2020-08-04: 3824.1448058583123
2020-08-05: 3097.421939524536
2020-08-06: 1886.9642826398108
2020-08-07: 1815.1763209915007
2020-08-08: 2562.8238502794734
2020-08-09: 3053.6055759078245
2020-08-10: 3434.0513277599143
2020-08-11: 3595.4522710078136
2020-08-12: 3303.919856297018
2020-08-13: 2845.5043791656594
2020-08-14: 2772.873805843756
2020-08-15: 3015.4468531483276
2020-08-16: 3287.6868516797745
2020-08-17: 3544.341483971542
2020-08-18: 3679.372218627404
2020-08-19: 3605.540971312554
2020-08-20: 3454.218144047411
2020-08-21: 3424.8384394436725
2020-08-22: 3524.502452669178
2020-08-23: 3680.0231984361476
2020-08-24: 3847.1765716031005
2020-08-25: 3960.14

2020-08-01: 671.7792392809029
2020-08-02: 739.2278288298077
2020-08-03: 775.3725492119611
2020-08-04: 764.0819012503576
2020-08-05: 581.2874758991
2020-08-06: 405.73206915740326
2020-08-07: 437.2665383086944
2020-08-08: 650.6381190493136
2020-08-09: 744.6203552276346
2020-08-10: 807.0973900009044
2020-08-11: 816.679172842615
2020-08-12: 749.9515032121715
2020-08-13: 681.4150981263421
2020-08-14: 698.6355295925526
2020-08-15: 780.0812966956229
2020-08-16: 847.109252399113
2020-08-17: 900.7106899495619
2020-08-18: 923.853242895752
2020-08-19: 910.9674719398951
2020-08-20: 894.4998047680737
2020-08-21: 909.7123240656414
2020-08-22: 951.6551021596707
2020-08-23: 998.3006234958127
2020-08-24: 1041.4086105621618
2020-08-25: 1070.1334132748366
2020-08-26: 1082.1080885278734
2020-08-27: 1090.6559296973041
2020-08-28: 1095.8746809994072
2020-08-29: 1121.9844450329642
2020-08-30: 1155.4655185272018
2020-08-31: 1190.0298498096731

Predicting for Belarus__nan
2020-08-01: 1175.0236222659491
2020-08

2020-08-02: 277.4908487812172
2020-08-03: 221.72910215197408
2020-08-04: 354.4628358786764
2020-08-05: 691.751420356524
2020-08-06: 597.2514573903062
2020-08-07: 438.3033238140434
2020-08-08: 505.99503761352275
2020-08-09: 460.47550990068277
2020-08-10: 444.71849432222376
2020-08-11: 532.1424166534575
2020-08-12: 656.3024620085095
2020-08-13: 650.6928631957702
2020-08-14: 625.6819070932112
2020-08-15: 639.4641782052519
2020-08-16: 628.5520309886377
2020-08-17: 634.4809785906174
2020-08-18: 684.8477814850908
2020-08-19: 743.3020009612529
2020-08-20: 763.8787685557198
2020-08-21: 774.6305283094378
2020-08-22: 788.108996687591
2020-08-23: 794.8105525377107
2020-08-24: 810.5407194960923
2020-08-25: 844.4338412656837
2020-08-26: 881.8702845043772
2020-08-27: 908.7550359976009
2020-08-28: 933.1432790799289
2020-08-29: 953.560892268582
2020-08-30: 971.218758558911
2020-08-31: 992.9664431886719

Predicting for Brazil__Espirito Santo
2020-08-01: 985.2218967261937
2020-08-02: 606.5868817495923
2

2020-08-01: 661.1569190985516
2020-08-02: 334.7120316112498
2020-08-03: 267.36121416128134
2020-08-04: 273.17129637459936
2020-08-05: 458.2625478806988
2020-08-06: 506.74753577857996
2020-08-07: 416.20779579243754
2020-08-08: 557.4177909548038
2020-08-09: 490.1516308922385
2020-08-10: 457.73142705899926
2020-08-11: 475.43845294083013
2020-08-12: 548.9227210758652
2020-08-13: 573.8766384518038
2020-08-14: 585.2807375188253
2020-08-15: 630.8849317854821
2020-08-16: 625.2004142207721
2020-08-17: 622.083876856164
2020-08-18: 641.2507142096891
2020-08-19: 677.5045771186712
2020-08-20: 702.5122830124064
2020-08-21: 727.8976061969992
2020-08-22: 756.2202375803098
2020-08-23: 769.4751792069216
2020-08-24: 781.8146330956342
2020-08-25: 802.4962871155819
2020-08-26: 829.1449513457843
2020-08-27: 854.6766537872794
2020-08-28: 883.9284948245603
2020-08-29: 910.9123506214133
2020-08-30: 932.5832009288533
2020-08-31: 953.3352923375508

Predicting for Brazil__Piaui
2020-08-01: 293.5561588364309
2020-

...

2020-08-03: 1138.3376292816665
2020-08-04: 1571.6708871673054
2020-08-05: 2392.071518209231
2020-08-06: 1235.5159041553916
2020-08-07: 1094.9517848019732
2020-08-08: 1596.252844740135
2020-08-09: 1900.1859003172103
2020-08-10: 1649.8405526210145
2020-08-11: 1892.3001982787619
2020-08-12: 2056.0424343498034
2020-08-13: 1740.7056536219538
2020-08-14: 1686.1444380663925
2020-08-15: 1863.7162918368804
2020-08-16: 1969.5457891282063
2020-08-17: 1973.0304882614223
2020-08-18: 2099.3806406580484
2020-08-19: 2156.9137397127597
2020-08-20: 2083.7737145569054
2020-08-21: 2082.23129703053
2020-08-22: 2157.009606430904
2020-08-23: 2217.7405238569704
2020-08-24: 2267.958319716586
2020-08-25: 2347.4191767074103
2020-08-26: 2393.792660929381
2020-08-27: 2399.9684287510504
2020-08-28: 2418.1046389037488
2020-08-29: 2465.291270441008
2020-08-30: 2514.8261597132946
2020-08-31: 2569.055246002161

Predicting for United States__Kentucky
2020-08-01: 2254.3439557973416
2020-08-02: 2731.89648601619
2020-08-03

2020-08-26: 3488.5376678851726
2020-08-27: 3470.3151701265633
2020-08-28: 3465.5535168383362
2020-08-29: 3513.8032023915607
2020-08-30: 3608.438449672117
2020-08-31: 3718.8190454964865

Predicting for United States__Mississippi
2020-08-01: 1321.5833899848335
2020-08-02: 1800.6266678731781
2020-08-03: 1970.1536409720584
2020-08-04: 1793.5510111515518
2020-08-05: 1587.0243986288096
2020-08-06: 981.4723542086571
2020-08-07: 961.9941547420539
2020-08-08: 1384.221149219115
2020-08-09: 1683.9086010653668
2020-08-10: 1840.9588073870564
2020-08-11: 1857.7619232961583
2020-08-12: 1740.5646376030563
2020-08-13: 1517.8419282559944
2020-08-14: 1496.3396361203472
2020-08-15: 1647.759699343169
2020-08-16: 1809.8692211077569
2020-08-17: 1930.1969924128566
2020-08-18: 1983.7154638669754
2020-08-19: 1953.6136865666822
2020-08-20: 1884.1149698354566
2020-08-21: 1881.4170321521165
2020-08-22: 1950.3487811760829
2020-08-23: 2044.0252011751186
2020-08-24: 2131.298863292589
2020-08-25: 2188.8438107786706
20

2020-08-01: 6361.02250762834
2020-08-02: 7947.644454048668
2020-08-03: 8883.309834772639
2020-08-04: 8818.640494711724
2020-08-05: 7413.236007884818
2020-08-06: 4433.019678889381
2020-08-07: 4312.297704060462
2020-08-08: 6214.259297090315
2020-08-09: 7374.042383400983
2020-08-10: 8182.168761701602
2020-08-11: 8429.363119326075
2020-08-12: 7767.856196789364
2020-08-13: 6644.740542055995
2020-08-14: 6490.8368749768415
2020-08-15: 7106.504034465466
2020-08-16: 7750.814340361554
2020-08-17: 8309.178089098188
2020-08-18: 8574.517044927432
2020-08-19: 8378.96419123296
2020-08-20: 7995.569103904232
2020-08-21: 7923.093076017542
2020-08-22: 8165.648151291445
2020-08-23: 8525.463666744443
2020-08-24: 8892.473360568776
2020-08-25: 9127.520439110971
2020-08-26: 9150.476299286753
2020-08-27: 9078.370895533599
2020-08-28: 9099.87189727279
2020-08-29: 9250.057513537904
2020-08-30: 9484.31911496367
2020-08-31: 9744.079760495943

Predicting for United States__Oklahoma
2020-08-01: 1774.7441463963473
20

2020-08-02: 89.73566659217789
2020-08-03: 85.70196963676723
2020-08-04: 92.63799662587175
2020-08-05: 100.77376372398166
2020-08-06: 99.9078789125734
2020-08-07: 112.22683835569559
2020-08-08: 143.0796969169696
2020-08-09: 162.4067530017812
2020-08-10: 173.25663349821588
2020-08-11: 184.9148404625875
2020-08-12: 194.9253161964882
2020-08-13: 202.90781171668945
2020-08-14: 217.1549941391524
2020-08-15: 237.47965652475733
2020-08-16: 255.34215017410602
2020-08-17: 270.57873718715393
2020-08-18: 285.0963184173035
2020-08-19: 298.2884845932538
2020-08-20: 311.1106724490219
2020-08-21: 326.59908018368793
2020-08-22: 344.485934689598
2020-08-23: 362.2206338139862
2020-08-24: 379.29961048857433
2020-08-25: 395.8771092531217
2020-08-26: 411.78701946427555
2020-08-27: 427.6964817927211
2020-08-28: 437.60982226860824
2020-08-29: 453.19989109111714
2020-08-30: 470.00823121863846
2020-08-31: 487.29707328123175

Predicting for United States__Vermont
2020-08-01: 118.89519046859036
2020-08-02: 148.94

2020-08-29: 430.31300610551136
2020-08-30: 443.05597980043297
2020-08-31: 457.3901305947075

Predicting for Vietnam__nan
2020-08-01: 48.26725000199204
2020-08-02: 66.00034339735062
2020-08-03: 76.57869033390268
2020-08-04: 84.55160876337096
2020-08-05: 91.38431255009073
2020-08-06: 94.28368817719638
2020-08-07: 105.10481544591408
2020-08-08: 128.7371998498786
2020-08-09: 146.4978439747198
2020-08-10: 160.99025384393997
2020-08-11: 173.51401194873281
2020-08-12: 184.114331504608
2020-08-13: 193.49415979203428
2020-08-14: 206.77072984235815
2020-08-15: 224.33656018730508
2020-08-16: 241.28947851138463
2020-08-17: 257.23189674895264
2020-08-18: 272.1203572439672
2020-08-19: 285.8094952292378
2020-08-20: 299.14283170416843
2020-08-21: 314.12861409674804
2020-08-22: 330.8723660940963
2020-08-23: 348.0032385990679
2020-08-24: 365.02510987096923
2020-08-25: 381.6475754120592
2020-08-26: 397.74525735393047
2020-08-27: 413.7834965243021
2020-08-28: 449.8209498873679
2020-08-29: 474.459654979469

In [35]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,54.448817
214,Aruba,NaN,2020-08-02,74.420533
215,Aruba,NaN,2020-08-03,83.765057
216,Aruba,NaN,2020-08-04,89.706375
217,Aruba,NaN,2020-08-05,95.334698


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [36]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [ ]:
!head predictions/2020-08-01_2020-08-04.csv

# Test cases
We can generate a prediction file. Let's validate a few cases...

In [ ]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")